# 第 6 章：构建 AI 智能体的其它 API 选项

> 本笔记文件需要与《LangGraph实战》的第 6 章的内容配套使用。

在构建智能体的道路上，框架赋予我们多种途径，如同百川汇流，殊途同归。关键在于洞悉每条路径的特性，权衡利弊，最终择取那条最能释放创造力、最能达成目标的通途。

本章将深入探索 LangGraph 框架的核心 —— 其丰富而强大的 API 工具箱。为了满足不同开发场景和开发者偏好，LangGraph 提供了多种 API 接口：

- **`create_react_agent`**: 预构建 API，快速启动按钮，迅速搭建功能完备的 ReAct 智能体
- **Functional API**: 以函数为中心的 API，通过 `@entrypoint` 和 `@task` 装饰器，将 LangGraph 的核心功能融入熟悉的函数式编程范式
- **Graph API**: 核心 API 范式，通过显式定义节点和边构建任意复杂度的智能体架构

### 🚀 环境准备

首先加载必要的环境变量配置：

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## 6.1 create_react_agent 

LangGraph 作为 AI 智能体开发框架，在灵活性和功能性方面展现出独特优势。该框架不仅支持高度定制化的智能体架构开发，同时为快速原型设计和开发者入门提供了便捷工具。其中，`create_react_agent` 函数作为预构建组件，实现了 ReAct（推理—
行动）模式的快速部署，显著降低了开发门槛。本节将演示 `create_react_agent` 的核心功能、自定义选项，以及如何在更广泛的 LangChain 生态系统中无缝集成。

##### 示例 6-1：`create_react_agent` 的基本用法

让我们从最简单的示例开始，构建一个能够查询天气信息的智能体：

In [1]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

# 初始化语言模型
model = ChatOpenAI(model="Qwen/Qwen3-8B", temperature=0)

# 定义一个简单的工具来获取天气信息
@tool
def get_weather(city: str):
    """Use this tool to get the weather information for a city."""
    if city.lower() in ["nyc", "new york"]:
        return "It might be cloudy in New York."
    elif city.lower() in ["sf", "san francisco"]:
        return "It's always sunny in San Francisco."
    else:
        return "Unable to get the weather information for this city."

tools = [get_weather]

# 创建 ReAct 智能体
graph = create_react_agent(model, tools=tools)

# 调用智能体
inputs = {"messages": [("user", "How's the weather in sf?")]}
response = graph.invoke(inputs)
print(response['messages'][-1].content)



The weather in San Francisco is currently sunny! 🌞 While the city is known for its foggy days, it's great to hear it's clear up today. Let me know if you'd like more details!


**💡 核心概念解析**：

- **模型绑定**: `create_react_agent` 自动将工具绑定到语言模型
- **工具定义**: 使用 `@tool` 装饰器将 Python 函数转换为 LangChain 工具
- **图构建**: 内部自动构建 ReAct 工作流程图，包含 agent 和 tools 节点
- **消息处理**: 支持标准的消息格式输入和输出

##### 示例 6-2：使用自定义系统提示

通过 `prompt` 参数，我们可以自定义智能体的行为和响应风格：

In [3]:
# 定义用于中文回复的自定义系统提示
chinese_prompt = "请用中文回复所有问题"

# 使用自定义提示创建 ReAct 智能体
graph_chinese = create_react_agent(model, tools=tools, prompt=chinese_prompt)

# 示例调用
inputs = {"messages": [("user", "How's the weather in nyc?")]}
response_chinese = graph_chinese.invoke(inputs)
print(response_chinese['messages'][-1].content)



紐約目前可能有雲，天氣較為陰沉。


**💡 提示定制的价值**：

- **语言控制**: 指定回复语言
- **角色设定**: 定义智能体的角色和个性
- **行为约束**: 设置特定的行为规则和限制
- **领域适应**: 为特定应用领域提供上下文

##### 示例 6-3：添加对话记忆功能

通过 `checkpointer` 参数，智能体可以跨多轮对话维护上下文：

In [4]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# ... 定义模型和工具（与之前相同）...

# 初始化内存检查点
memory = MemorySaver()

# 创建具有记忆功能的 ReAct 智能体
graph_with_memory = create_react_agent(model, tools=tools, checkpointer=memory)

# 首次交互
config = {"configurable": {"thread_id": "user_thread_1"}} # 唯一线程 ID
inputs_1 = {"messages": [("user", "How's the weather in sf?")]}
response_1 = graph_with_memory.invoke(inputs_1, config=config)
print(response_1['messages'][-1].content)

# 同一线程中的第二次交互 - 智能体记住上下文
inputs_2 = {"messages": [("user", "How is chicago?")]}
response_2 = graph_with_memory.invoke(inputs_2, config=config)
print(response_2['messages'][-1].content)



The current weather in San Francisco is sunny. 🌞 San Francisco is known for its mild climate and occasional fog, but today the skies are clear. Hope you enjoy the sunshine!


I'm sorry, but I couldn't retrieve the weather information for Chicago. It's possible there was a temporary issue or the city name isn't recognized. Let me know if you'd like to check another city or need further assistance! ☁️☀️


**💡 记忆机制解析**：

- **线程隔离**: 通过 `thread_id` 区分不同的对话会话
- **状态持久化**: `MemorySaver` 在内存中保存对话历史
- **上下文连续性**: 智能体能够理解跨轮次的对话上下文
- **存储选择**: 生产环境可选择 `RedisSaver` 或 `SQLiteSaver`

##### 示例 6-4：实现人机环路工作流程

通过 `interrupt_before` 参数，我们可以在关键节点暂停智能体执行，实现人工监督：

In [5]:
# 创建启用人机环路的 ReAct 智能体 - 在工具调用前中断
graph_hitl = create_react_agent(model, tools=tools, interrupt_before=["tools"], checkpointer=memory)

# 首次交互 - 智能体将在工具调用之前暂停
config_hitl = {"configurable": {"thread_id": "user_thread_hitl"}}
inputs_hitl = {"messages": [("user", "How's the weather in sf?")]}

print("执行智能体（将在工具调用前暂停）...")
stream = graph_hitl.stream(inputs_hitl, config=config_hitl, stream_mode="values")
for output in stream:
    if 'messages' in output and output['messages']:
        last_message = output['messages'][-1]
        print(f"消息类型: {type(last_message).__name__}")
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            print(f"计划调用工具: {last_message.tool_calls}")
            print(">>> 智能体已暂停，等待人工审核 <<<")
            break
        else:
            print(f"内容: {last_message.content}")

# 检查当前状态
print("\n当前智能体状态:")
current_state = graph_hitl.get_state(config_hitl)
print(f"下一个节点: {current_state.next}")
print(f"消息数量: {len(current_state.values['messages'])}")

执行智能体（将在工具调用前暂停）...
消息类型: HumanMessage
内容: How's the weather in sf?
消息类型: AIMessage
计划调用工具: [{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': '0198ebdf1c7d7d658f8d063b41a21d79', 'type': 'tool_call'}]
>>> 智能体已暂停，等待人工审核 <<<

当前智能体状态:
下一个节点: ('tools',)
消息数量: 2


**💡 人机环路的核心价值**：

- **安全控制**: 在执行敏感操作前引入人工审核
- **质量保证**: 确保工具调用的准确性和合理性
- **风险管理**: 防止自动化系统执行不当操作
- **学习机会**: 通过人工反馈改进智能体决策

##### 示例 6-5：结构化输出响应

使用 `response_format` 参数，智能体可以返回符合预定义结构的响应：

In [ ]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# 定义结构化输出的 Pydantic 模型
class WeatherResponse(BaseModel):
    """Respond with a weather description."""
    city: str = Field(description="The city name")
    conditions: str = Field(description="Weather conditions description")
    temperature: str = Field(description="Temperature information")
    confidence: str = Field(description="Confidence level of the weather data")

# 注意：结构化输出功能需要支持原生结构化输出的模型
# 这里我们演示概念，实际使用时请确保模型支持，建议使用 OpenAI 模型实验
try:
    # 创建启用结构化输出的 ReAct 智能体
    graph_structured = create_react_agent(model, tools=tools, response_format=WeatherResponse)
    
    # 示例调用
    inputs_structured = {"messages": [("user", "How's the weather in sf?")]}
    response_structured = graph_structured.invoke(inputs_structured)
    
    # 访问结构化响应
    if "structured_response" in response_structured:
        structured_data = response_structured["structured_response"]
        print("结构化天气响应:")
        print(f"城市: {structured_data.city}")
        print(f"天气状况: {structured_data.conditions}")
        print(f"温度: {structured_data.temperature}")
        print(f"置信度: {structured_data.confidence}")
    else:
        print("常规响应:", response_structured['messages'][-1].content)
except Exception as e:
    print(f"结构化输出示例跳过 (需要兼容模型): {e}")
    print("使用常规输出进行演示...")
    regular_response = graph.invoke({"messages": [("user", "How's the weather in sf?")]})
    print(regular_response['messages'][-1].content)

**💡 结构化输出的应用场景**：

- **API 集成**: 与下游系统的标准化数据交换
- **数据处理**: 便于程序化处理和分析
- **用户界面**: 为前端提供结构化的展示数据
- **质量控制**: 确保输出格式的一致性和完整性

## 6.2 Functional API：拥抱工作流设计的简洁性和熟悉性

Functional API 作为 Graph API 的补充，旨在提供一种更简洁、更直观、更贴近传统编程习惯的工作流构建方式。它在保留 LangGraph 核心功能的同时，让开发者能够使用熟悉的 Python 函数和装饰器来定义工作流。

### 设计理念
- **降低学习门槛**: 对新手开发人员更加友好
- **代码集成性**: 与现有 Python 代码库无缝集成
- **快速开发**: 减少样板代码，加速原型设计
- **功能完整性**: 支持持久性、内存、人机环路和流式传输

### 核心组件：`@entrypoint` 和 `@task`

Functional API 基于两个基本装饰器构建：
- **`@entrypoint`**: 定义工作流的入口点和边界
- **`@task`**: 封装独立的工作单元

##### 示例 6-6：具有参数用法的复杂 `@entrypoint` 函数

展示 `@entrypoint` 装饰器的高级用法，包括所有可注入参数：

In [9]:
from langgraph.func import entrypoint
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore, BaseStore
from langgraph.types import StreamWriter
from langchain_core.runnables import RunnableConfig
from typing import Any

store = InMemoryStore()
checkpointer = MemorySaver()

@entrypoint(checkpointer=checkpointer, store=store)
def my_workflow(
    user_input: dict, # 输入参数
    *,
    previous: Any = None, # 用于先前状态的可注入参数
    store: BaseStore,    # 用于长期内存存储的可注入参数
    writer: StreamWriter, # 用于自定义流写入器的可注入参数
    config: RunnableConfig # 用于运行时配置的可注入参数
) -> str:
    """一个复杂的演示入口点参数的工作流。"""
    thread_id = config.get("configurable", {}).get("thread_id", "unknown")
    
    # 向客户端流式传输自定义信息
    writer(f"工作流 '{thread_id}' 已启动")
    
    # 检查先前状态
    if previous:
        writer(f"发现先前状态: {previous}")
    else:
        writer("这是首次执行")
    
    # 访问长期存储
    try:
        stored_data = store.get(("demo", "counter"), "value")
        if stored_data:
            counter = stored_data.value + 1
        else:
            counter = 1
        store.put(("demo", "counter"), "value", counter)
        writer(f"更新计数器: {counter}")
    except Exception as e:
        writer(f"存储操作错误: {e}")
        counter = 1
    
    result = f"工作流处理的输入：{user_input}, 执行次数: {counter}"
    writer("工作流处理完成")
    
    return result

# 示例调用
config = {"configurable": {"thread_id": "complex_workflow_1"}}
print("=== 使用 invoke 执行 ===")
result = my_workflow.invoke({"message": "Hello"}, config)
print(f"结果: {result}")
print()

# 使用流式执行查看中间过程
print("=== 使用 stream 执行 ===")
for chunk in my_workflow.stream({"message": "World"}, config, stream_mode=["custom", "updates"]):
    print(f"流式数据: {chunk}")

=== 使用 invoke 执行 ===
结果: 工作流处理的输入：{'message': 'Hello'}, 执行次数: 1

=== 使用 stream 执行 ===
流式数据: ('custom', "工作流 'complex_workflow_1' 已启动")
流式数据: ('custom', "发现先前状态: 工作流处理的输入：{'message': 'Hello'}, 执行次数: 1")
流式数据: ('custom', '更新计数器: 2')
流式数据: ('custom', '工作流处理完成')
流式数据: ('updates', {'my_workflow': "工作流处理的输入：{'message': 'World'}, 执行次数: 2"})


**💡 `@entrypoint` 高级特性解析**：

- **可注入参数**: `previous`、`store`、`writer`、`config` 由运行时自动注入
- **状态持久化**: 通过 `checkpointer` 参数实现跨执行状态保存
- **长期存储**: `store` 参数提供持久化存储能力
- **实时流式传输**: `writer` 支持自定义数据流式输出
- **运行时配置**: `config` 提供执行时的配置信息访问

##### 示例 6-7：带有重试策略的 `@task` 函数

演示如何使用 `@task` 装饰器的重试功能来处理不可靠的操作：

In [10]:
from langgraph.func import task
from langgraph.types import RetryPolicy
import time
import random

# 定义重试策略
retry_policy = RetryPolicy(max_attempts=3, retry_on=TimeoutError)

@task(name="api_data_fetcher", retry_policy=retry_policy)
def fetch_api_data(api_endpoint: str) -> dict:
    """从带有重试策略的 API 端点获取数据的任务。"""
    print(f"尝试从 {api_endpoint} 获取数据...")
    time.sleep(random.uniform(0.1, 0.5)) # 模拟网络延迟
    
    if random.random() < 0.8: # 模拟 60% 的超时概率
        print("API 请求超时！")
        raise TimeoutError("API 请求超时")
    
    print("API 请求成功！")
    return {"status": "success", "data": f"来自 {api_endpoint} 的数据"}

# 在入口点内调用任务的示例
@entrypoint(checkpointer=MemorySaver())
def data_processing_workflow(endpoint_url: str) -> dict:
    """调用 fetch_api_data 任务的工作流。"""
    try:
        api_result = fetch_api_data(api_endpoint=endpoint_url).result()
        return {"workflow_result": "数据已处理", "api_response": api_result}
    except TimeoutError as e:
        return {"workflow_result": "处理失败", "error": str(e)}

# 测试重试机制
config = {"configurable": {"thread_id": "task_params_workflow_1"}}
print("测试带重试策略的任务...")
result = data_processing_workflow.invoke("https://api.example.com/data", config)
print(f"\n最终结果: {result}")

测试带重试策略的任务...
尝试从 https://api.example.com/data 获取数据...
API 请求超时！
尝试从 https://api.example.com/data 获取数据...
API 请求超时！
尝试从 https://api.example.com/data 获取数据...
API 请求成功！

最终结果: {'workflow_result': '数据已处理', 'api_response': {'status': 'success', 'data': '来自 https://api.example.com/data 的数据'}}


**💡 `@task` 重试机制的优势**：

- **容错能力**: 自动处理瞬时网络错误和服务不可用
- **指定异常**: `retry_on` 参数允许针对特定异常类型重试
- **自定义策略**: 可配置最大重试次数和重试条件
- **任务隔离**: 重试逻辑封装在任务级别，不影响整体工作流

##### 示例 6-8：包含控制流和任务调用的工作流逻辑

展示在 `@entrypoint` 中使用 Python 控制流和任务调用构建条件逻辑：

In [26]:
@task
def is_even(number: int) -> bool:
    """检查数字是否为偶数的任务。"""
    print(f"检查 {number} 是否为偶数...")
    return number % 2 == 0

@task
def multiply_by_two(number: int) -> int:
    """将数字乘以二的任务。"""
    print(f"将 {number} 乘以 2...")
    return number * 2

@task
def add_ten(number: int) -> int:
    """将数字加十的任务。"""
    print(f"将 {number} 加 10...")
    return number + 10

@entrypoint(checkpointer=MemorySaver())
def number_workflow(input_number: int) -> dict:
    """根据数字是奇数还是偶数处理数字的工作流。"""
    print(f"处理数字: {input_number}")
    
    # 检查是否为偶数
    is_even_result = is_even(input_number).result()
    
    if is_even_result:
        # 偶数：乘以2
        result = multiply_by_two(input_number).result()
        operation = "乘以2"
    else:
        # 奇数：加10
        result = add_ten(input_number).result()
        operation = "加10"
    
    return {
        "input": input_number,
        "is_even": is_even_result,
        "operation": operation,
        "result": result
    }

# 测试不同数字
config = {"configurable": {"thread_id": "number_workflow_1"}}

print("=== 测试偶数 ===")
result_even = number_workflow.invoke(4, config)
print(f"结果: {result_even}")
print()

print("=== 测试奇数 ===")
result_odd = number_workflow.invoke(5, config)
print(f"结果: {result_odd}")

=== 测试偶数 ===
处理数字: 4
检查 4 是否为偶数...
将 4 乘以 2...
结果: {'input': 4, 'is_even': True, 'operation': '乘以2', 'result': 8}

=== 测试奇数 ===
处理数字: 5
检查 5 是否为偶数...
将 5 加 10...
结果: {'input': 5, 'is_even': False, 'operation': '加10', 'result': 15}


**💡 控制流在 Functional API 中的应用**：

- **条件分支**: 使用标准 Python `if/else` 语句实现工作流分支
- **任务编排**: 通过 `.result()` 方法同步获取任务执行结果
- **状态传递**: 任务结果可作为后续任务的输入
- **灵活组合**: 可以根据业务逻辑动态组合不同的任务执行路径

##### 示例 6-9：使用 `invoke` 同步执行工作流

演示同步工作流执行的基本模式：

In [27]:
@task
def process_data(data: str) -> str:
    """处理数据的简单任务。"""
    time.sleep(0.1)  # 模拟处理时间
    return f"已处理: {data.upper()}"

@entrypoint(checkpointer=MemorySaver())
def simple_workflow(input_data: str) -> str:
    """简单的数据处理工作流。"""
    processed = process_data(input_data).result()
    return f"工作流完成: {processed}"

# 同步执行示例
config = {"configurable": {"thread_id": "sync_workflow_1"}}
print("执行同步工作流...")
result = simple_workflow.invoke("hello world", config)
print(f"同步结果: {result}")

# 多次调用同一工作流
inputs = ["apple", "banana", "cherry"]
for i, data in enumerate(inputs, 1):
    config["configurable"]["thread_id"] = f"sync_workflow_{i}"
    result = simple_workflow.invoke(data, config)
    print(f"调用 {i}: {result}")

执行同步工作流...
同步结果: 工作流完成: 已处理: HELLO WORLD
调用 1: 工作流完成: 已处理: APPLE
调用 2: 工作流完成: 已处理: BANANA
调用 3: 工作流完成: 已处理: CHERRY


**💡 同步执行的特点**：

- **阻塞执行**: `invoke` 方法会等待工作流完全执行完毕
- **简单直接**: 适合不需要实时反馈的场景
- **错误处理**: 执行过程中的异常会直接抛出
- **线程隔离**: 每个 `thread_id` 维护独立的状态

##### 示例 6-10：使用 `stream` 流式执行工作流

演示流式执行，提供实时执行进度反馈：

In [28]:
@task
def step_one(data: str) -> str:
    """第一个处理步骤。"""
    time.sleep(0.2)
    return f"步骤1完成: {data}"

@task
def step_two(data: str) -> str:
    """第二个处理步骤。"""
    time.sleep(0.2)
    return f"步骤2完成: {data}"

@task
def step_three(data: str) -> str:
    """第三个处理步骤。"""
    time.sleep(0.2)
    return f"步骤3完成: {data}"

@entrypoint(checkpointer=MemorySaver())
def multi_step_workflow(input_data: str) -> str:
    """多步骤工作流，展示流式执行。"""
    result1 = step_one(input_data).result()
    result2 = step_two(result1).result()
    result3 = step_three(result2).result()
    
    return f"最终结果: {result3}"

# 流式执行示例
config = {"configurable": {"thread_id": "stream_workflow_1"}}
print("执行流式工作流（观察执行进度）...")
print("=" * 50)

for i, chunk in enumerate(multi_step_workflow.stream("测试数据", config, stream_mode=["updates"]), 1):
    print(f"流式更新 {i}: {chunk}")
    time.sleep(0.1)  # 稍微延迟以便观察

print("=" * 50)
print("流式执行完成！")

执行流式工作流（观察执行进度）...
流式更新 1: ('updates', {'step_one': '步骤1完成: 测试数据'})
流式更新 2: ('updates', {'step_two': '步骤2完成: 步骤1完成: 测试数据'})
流式更新 3: ('updates', {'step_three': '步骤3完成: 步骤2完成: 步骤1完成: 测试数据'})
流式更新 4: ('updates', {'multi_step_workflow': '最终结果: 步骤3完成: 步骤2完成: 步骤1完成: 测试数据'})
流式执行完成！


**💡 流式执行的优势**：

- **实时反馈**: 用户可以看到工作流的执行进度
- **改善体验**: 长时间运行的任务不会让用户感到等待
- **早期发现**: 可以及时发现执行过程中的问题
- **灵活监控**: 支持多种流模式（updates、messages、custom）

##### 示例 6-11：使用 `Command` 对象恢复工作流

演示人机环路中断后的工作流恢复机制：

In [12]:
from langgraph.types import Command, interrupt

@task
def require_approval(request: str) -> str:
    """需要人工审核的任务。"""
    print(f"请求审核: {request}")
    
    # 中断工作流，等待人工输入
    human_feedback = interrupt(f"请审核以下请求: {request}")
    
    if human_feedback and "批准" in str(human_feedback):
        return f"请求已批准: {request}"
    else:
        return f"请求被拒绝: {request}"

@task
def process_approved_request(approved_request: str) -> str:
    """处理已批准的请求。"""
    time.sleep(0.1)
    return f"处理完成: {approved_request}"

@entrypoint(checkpointer=MemorySaver())
def approval_workflow(request: str) -> str:
    """需要审批的工作流。"""
    approval_result = require_approval(request).result()
    
    if "已批准" in approval_result:
        final_result = process_approved_request(approval_result).result()
        return final_result
    else:
        return f"工作流终止: {approval_result}"

# 演示中断和恢复
config = {"configurable": {"thread_id": "approval_workflow_1"}}
request_data = "创建新用户账户"

print("=== 第一阶段：执行工作流直到中断 ===")
try:
    # 首次执行，将在 interrupt 处暂停
    for chunk in approval_workflow.stream(request_data, config, stream_mode=["updates"]):
        print(f"执行更新: {chunk}")
except Exception as e:
    print(f"工作流已中断: {e}")

# 检查工作流状态
print("\n=== 检查工作流状态 ===")
state = approval_workflow.get_state(config)
print(f"工作流是否已中断: {state.next is not None}")
if hasattr(state, 'tasks') and state.tasks:
    if isinstance(state.tasks, tuple):
        print(f"等待中断的任务数量: {len(state.tasks)}")
        for i, task in enumerate(state.tasks):
            print(f"任务 {i+1}: {task}")
    elif hasattr(state.tasks, 'keys'):
        print(f"等待中断的任务: {list(state.tasks.keys())}")
    else:
        print(f"中断状态: {state.tasks}")

# 模拟人工审核后恢复
print("\n=== 第二阶段：提供人工反馈并恢复 ===")
try:
    # 使用 Command 对象提供人工反馈
    resume_command = Command(resume="批准请求")
    
    print("提供人工反馈并恢复工作流...")
    for chunk in approval_workflow.stream(resume_command, config, stream_mode=["updates"]):
        print(f"恢复执行: {chunk}")
    
    # 获取最终结果
    final_state = approval_workflow.get_state(config)
    if hasattr(final_state, 'values') and 'return_value' in final_state.values:
        print(f"\n最终结果: {final_state.values['return_value']}")
except Exception as e:
    print(f"恢复执行时出错: {e}")

=== 第一阶段：执行工作流直到中断 ===
请求审核: 创建新用户账户
执行更新: ('updates', {'__interrupt__': (Interrupt(value='请审核以下请求: 创建新用户账户', id='c6d55bc69a12e9beeb9c2842c724e19c'),)})

=== 检查工作流状态 ===
工作流是否已中断: True
等待中断的任务数量: 1
任务 1: PregelTask(id='3cf1d38b-ce07-0a90-82ad-4d6b4edc4aca', name='approval_workflow', path=('__pregel_pull', 'approval_workflow'), error=None, interrupts=(Interrupt(value='请审核以下请求: 创建新用户账户', id='c6d55bc69a12e9beeb9c2842c724e19c'),), state=None, result=None)

=== 第二阶段：提供人工反馈并恢复 ===
提供人工反馈并恢复工作流...
请求审核: 创建新用户账户
恢复执行: ('updates', {'require_approval': '请求已批准: 创建新用户账户'})
恢复执行: ('updates', {'process_approved_request': '处理完成: 请求已批准: 创建新用户账户'})
恢复执行: ('updates', {'approval_workflow': '处理完成: 请求已批准: 创建新用户账户'})


**💡 Command 恢复机制解析**：

- **中断函数**: `interrupt()` 可在任务中暂停工作流执行
- **状态保存**: 中断点的状态被自动保存到 checkpointer
- **人工介入**: 允许人类在关键决策点提供输入
- **无缝恢复**: 使用 `Command(resume=...)` 从中断点继续执行

##### 示例 6-12：在 Functional API 工作流中使用 LangChain LLM 集成

展示 Functional API 与 LangChain 组件的深度集成：

In [17]:
from langchain_openai import ChatOpenAI
from langgraph.func import task, entrypoint
from langgraph.checkpoint.memory import MemorySaver

llm = ChatOpenAI(model="Qwen/Qwen3-8B", temperature=0.7)

@task
def generate_response(user_query: str) -> str:
    """生成使用 LangChain LLM 的响应的任务。"""
    print(f"处理查询: {user_query}")
    response = llm.invoke(user_query)
    return response.content

@task
def analyze_sentiment(text: str) -> str:
    """分析文本情感。"""
    sentiment_prompt = f"分析以下文本的情感倾向，回答'积极'、'消极'或'中性'：{text}"
    response = llm.invoke(sentiment_prompt)
    return response.content.strip()

@task
def enhance_response(original_response: str, sentiment: str) -> str:
    """根据情感分析增强响应。"""
    if "积极" in sentiment:
        enhancement = "太好了！"
    elif "消极" in sentiment:
        enhancement = "我理解您的担忧。"
    else:
        enhancement = "谢谢您的问题。"
    
    return f"{enhancement} {original_response}"

@entrypoint(checkpointer=MemorySaver())
def smart_chatbot_workflow(query: str) -> dict:
    """智能聊天机器人工作流，具备情感分析能力。"""
    # 生成初始响应
    initial_response = generate_response(query).result()
    
    # 分析用户查询的情感
    user_sentiment = analyze_sentiment(query).result()
    
    # 根据情感增强响应
    enhanced_response = enhance_response(initial_response, user_sentiment).result()
    
    return {
        "user_query": query,
        "initial_response": initial_response,
        "detected_sentiment": user_sentiment,
        "final_response": enhanced_response
    }

# 测试智能聊天机器人
config = {"configurable": {"thread_id": "chatbot_1"}}

# 测试不同情感的查询
test_queries = [
    "我今天心情很好！你能告诉我一个笑话吗？",
    "我对产品的质量很不满意",
    "请问你们的营业时间是什么？"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n=== 测试查询 {i} ===")
    print(f"用户输入: {query}")
    
    config["configurable"]["thread_id"] = f"chatbot_{i}"
    result = smart_chatbot_workflow.invoke(query, config)
    
    print(f"检测情感: {result['detected_sentiment']}")
    print(f"最终回复: {result['final_response']}")


=== 测试查询 1 ===
用户输入: 我今天心情很好！你能告诉我一个笑话吗？
处理查询: 我今天心情很好！你能告诉我一个笑话吗？
检测情感: 积极
最终回复: 太好了！ 

当然可以！这里有一个轻松的小笑话：

**为什么企鹅不会迷路？**  
因为它们有“企”（启）和“鹅”（鹅），组合起来就是“启鹅”——所以它们总能找到方向！ 😄

（谐音梗+动物梗，希望能让您会心一笑～）

=== 测试查询 2 ===
用户输入: 我对产品的质量很不满意
处理查询: 我对产品的质量很不满意
检测情感: 消极
最终回复: 我理解您的担忧。 

很抱歉听到您对产品质量不满意，这确实会让人感到困扰。为了更好地帮助您解决问题，请您提供更多详细信息，例如：

1. **产品名称或型号**（如：手机、家电、电子产品等）  
2. **购买渠道**（如：电商平台、线下门店、官网等）  
3. **具体问题描述**（如：功能故障、外观瑕疵、包装损坏等）  
4. **是否已联系客服**（如：是否有售后沟通记录）  

根据您提供的信息，我会尽力为您提供针对性的解决方案，例如：  
- 检查产品是否符合质量标准  
- 指导您如何申请退换货或维修  
- 协助联系相关客服或技术支持团队  

同时，请您注意保留以下凭证：  
- 产品照片/视频（展示问题细节）  
- 发票或订单号  
- 与客服沟通的记录（如聊天截图、邮件等）  

我们非常重视您的反馈，期待尽快为您解决！ 🌟

=== 测试查询 3 ===
用户输入: 请问你们的营业时间是什么？
处理查询: 请问你们的营业时间是什么？
检测情感: 中性

这句话是一个中性的问题，询问营业时间本身不带有积极或消极的情感色彩。它属于客观信息的查询，没有表达满意、不满、喜悦、愤怒等情绪。
最终回复: 太好了！ 

您好！关于营业时间，我需要您提供更具体的信息，例如您想了解的是哪个店铺、商场或服务的营业时间？不同的场所可能有不同的营业安排，比如有的商场是早9点到晚9点，有的店铺可能有午休或周末休息时间。如果您能说明具体地点或服务类型，我可以更好地为您解答哦！ 😊

如果无法提供具体信息，建议您直接联系相关商家或查看其官方公告，以获取最准确的营业时间。


**💡 LangChain 集成的价值**：

- **组件复用**: 充分利用 LangChain 生态系统的现有组件
- **模型抽象**: 通过统一接口使用不同的语言模型
- **功能增强**: LangGraph 的状态管理增强了 LangChain 的能力
- **生态协同**: 两个框架的优势相互补充

##### 示例 6-13：在 Functional API 工作流中实现任务的并行执行

展示如何并行执行多个独立任务以提高性能：

In [ ]:
@task
def fetch_user_info(user_id: str) -> dict:
    """获取用户信息的任务。"""
    time.sleep(0.3)  # 模拟 API 调用延迟
    return {
        "user_id": user_id,
        "name": f"User-{user_id}",
        "email": f"{user_id}@example.com"
    }

@task
def fetch_user_orders(user_id: str) -> list:
    """获取用户订单的任务。"""
    time.sleep(0.4)  # 模拟数据库查询延迟
    return [
        {"order_id": f"ord-{user_id}-001", "amount": 99.99},
        {"order_id": f"ord-{user_id}-002", "amount": 149.99}
    ]

@task
def fetch_user_preferences(user_id: str) -> dict:
    """获取用户偏好的任务。"""
    time.sleep(0.2)  # 模拟缓存查询延迟
    return {
        "theme": "dark",
        "language": "zh-CN",
        "notifications": True
    }

@task
def aggregate_user_data(user_info: dict, orders: list, preferences: dict) -> dict:
    """聚合用户数据的任务。"""
    return {
        "user_profile": user_info,
        "order_history": orders,
        "user_preferences": preferences,
        "total_orders": len(orders),
        "total_spent": sum(order["amount"] for order in orders)
    }

@entrypoint(checkpointer=MemorySaver())
def parallel_user_data_workflow(user_id: str) -> dict:
    """并行获取用户数据的工作流。"""
    print(f"开始并行获取用户 {user_id} 的数据...")
    
    # 并行启动所有数据获取任务
    user_info_future = fetch_user_info(user_id)
    orders_future = fetch_user_orders(user_id)
    preferences_future = fetch_user_preferences(user_id)
    
    print("所有数据获取任务已启动，等待结果...")
    
    # 等待所有任务完成并获取结果
    user_info = user_info_future.result()
    orders = orders_future.result()
    preferences = preferences_future.result()
    
    print("所有数据已获取，开始聚合...")
    
    # 聚合数据
    aggregated_data = aggregate_user_data(user_info, orders, preferences).result()
    
    return aggregated_data

# 测试并行执行
import time

config = {"configurable": {"thread_id": "parallel_workflow_1"}}
user_id = "12345"

start_time = time.time()
print("=== 并行执行工作流 ===")
result = parallel_user_data_workflow.invoke(user_id, config)
end_time = time.time()

print(f"\n执行耗时: {end_time - start_time:.2f} 秒")
print(f"用户档案: {result['user_profile']}")
print(f"订单总数: {result['total_orders']}")
print(f"总消费: ${result['total_spent']:.2f}")
print(f"用户偏好: {result['user_preferences']}")

# 对比：如果是串行执行会花费更长时间
# (0.3 + 0.4 + 0.2 = 0.9秒 vs 并行执行约0.4秒)

=== 并行执行工作流 ===
开始并行获取用户 12345 的数据...
所有数据获取任务已启动，等待结果...
所有数据已获取，开始聚合...

执行耗时: 0.41 秒
用户档案: {'user_id': '12345', 'name': 'User-12345', 'email': '12345@example.com'}
订单总数: 2
总消费: $249.98
用户偏好: {'theme': 'dark', 'language': 'zh-CN', 'notifications': True}


**💡 并行执行的性能优势**：

- **时间效率**: 多个独立任务可同时执行，显著减少总耗时
- **资源利用**: 充分利用系统的并发处理能力
- **I/O 优化**: 特别适用于网络请求、数据库查询等 I/O 密集型任务
- **简单实现**: 无需复杂的并发编程，框架自动处理并行执行

##### 示例 6-14：在 Functional API 中调用子图

展示如何组合和嵌套工作流，实现模块化设计：

In [18]:
# 定义子工作流
@entrypoint(checkpointer=MemorySaver())
def data_validation_workflow(data: dict) -> dict:
    """数据验证子工作流。"""
    print(f"验证数据: {list(data.keys())}")
    
    validation_results = {
        "is_valid": True,
        "errors": [],
        "warnings": []
    }
    
    # 模拟数据验证逻辑
    if "name" not in data:
        validation_results["errors"].append("缺少 name 字段")
        validation_results["is_valid"] = False
    
    if "email" in data and "@" not in data["email"]:
        validation_results["errors"].append("邮箱格式无效")
        validation_results["is_valid"] = False
    
    if "age" in data and data["age"] < 0:
        validation_results["warnings"].append("年龄为负数")
    
    return {
        "original_data": data,
        "validation": validation_results
    }

@entrypoint(checkpointer=MemorySaver())
def data_processing_workflow(data: dict) -> dict:
    """数据处理子工作流。"""
    print(f"处理数据: {data}")
    
    processed_data = data.copy()
    
    # 模拟数据处理逻辑
    if "name" in processed_data:
        processed_data["name"] = processed_data["name"].title()
    
    if "email" in processed_data:
        processed_data["email"] = processed_data["email"].lower()
    
    processed_data["processed_at"] = "2024-01-01 12:00:00"
    
    return {
        "original_data": data,
        "processed_data": processed_data
    }

# 主工作流调用子工作流
@entrypoint(checkpointer=MemorySaver())
def master_data_workflow(raw_data: dict) -> dict:
    """主数据工作流，协调多个子工作流。"""
    print("=== 开始主数据工作流 ===")
    
    # 第一步：数据验证
    print("步骤 1: 数据验证")
    validation_result = data_validation_workflow.invoke(raw_data)
    
    if not validation_result["validation"]["is_valid"]:
        return {
            "status": "failed",
            "reason": "数据验证失败",
            "errors": validation_result["validation"]["errors"],
            "raw_data": raw_data
        }
    
    # 第二步：数据处理（仅在验证通过时执行）
    print("步骤 2: 数据处理")
    processing_result = data_processing_workflow.invoke(raw_data)
    
    return {
        "status": "success",
        "validation_result": validation_result["validation"],
        "processed_data": processing_result["processed_data"],
        "original_data": raw_data
    }

# 测试子工作流组合
config = {"configurable": {"thread_id": "master_workflow_1"}}

# 测试有效数据
print("=== 测试有效数据 ===")
valid_data = {
    "name": "john doe",
    "email": "John.Doe@Example.COM",
    "age": 30
}

result_valid = master_data_workflow.invoke(valid_data, config)
print(f"处理结果: {result_valid['status']}")
if result_valid['status'] == 'success':
    print(f"处理后数据: {result_valid['processed_data']}")
print()

# 测试无效数据
print("=== 测试无效数据 ===")
invalid_data = {
    "email": "invalid-email",  # 无效邮箱
    "age": -5  # 负数年龄
}

config["configurable"]["thread_id"] = "master_workflow_2"
result_invalid = master_data_workflow.invoke(invalid_data, config)
print(f"处理结果: {result_invalid['status']}")
if result_invalid['status'] == 'failed':
    print(f"失败原因: {result_invalid['reason']}")
    print(f"错误信息: {result_invalid['errors']}")

=== 测试有效数据 ===
=== 开始主数据工作流 ===
步骤 1: 数据验证
验证数据: ['name', 'email', 'age']
步骤 2: 数据处理
处理数据: {'name': 'john doe', 'email': 'John.Doe@Example.COM', 'age': 30}
处理结果: success
处理后数据: {'name': 'John Doe', 'email': 'john.doe@example.com', 'age': 30, 'processed_at': '2024-01-01 12:00:00'}

=== 测试无效数据 ===
=== 开始主数据工作流 ===
步骤 1: 数据验证
验证数据: ['email', 'age']
处理结果: failed
失败原因: 数据验证失败
错误信息: ['缺少 name 字段', '邮箱格式无效']


**💡 子工作流组合的优势**：

- **模块化设计**: 将复杂工作流分解为可重用的子工作流
- **职责分离**: 每个子工作流专注于特定功能
- **易于测试**: 可以独立测试每个子工作流
- **代码复用**: 子工作流可在多个主工作流中重复使用
- **状态隔离**: 每个工作流可以有独立的状态管理

##### 示例 6-15：在 Functional API 中演示自定义数据流式传输

展示如何使用 `writer` 参数实现自定义数据的实时流式传输：

In [19]:
from langgraph.types import StreamWriter

@task
def analyze_text(text: str) -> dict:
    """分析文本的任务。"""
    time.sleep(0.5)  # 模拟分析时间
    
    return {
        "length": len(text),
        "words": len(text.split()),
        "sentences": text.count('.') + text.count('!') + text.count('?'),
        "paragraphs": text.count('\n\n') + 1
    }

@task
def generate_summary(analysis: dict) -> str:
    """生成摘要的任务。"""
    time.sleep(0.3)  # 模拟生成时间
    
    return f"文本包含 {analysis['words']} 个单词，{analysis['sentences']} 个句子，{analysis['paragraphs']} 个段落。"

@entrypoint(checkpointer=MemorySaver())
def text_analysis_workflow(
    text: str, 
    *,
    writer: StreamWriter
) -> dict:
    """文本分析工作流，带有实时进度反馈。"""
    writer("开始文本分析工作流")
    writer(f"输入文本长度: {len(text)} 字符")
    
    # 第一阶段：文本分析
    writer("阶段 1: 正在分析文本结构...")
    analysis = analyze_text(text).result()
    writer(f"分析完成: 发现 {analysis['words']} 个单词")
    
    # 第二阶段：生成摘要
    writer("阶段 2: 正在生成摘要...")
    summary = generate_summary(analysis).result()
    writer("摘要生成完成")
    
    # 最终结果
    result = {
        "text_analysis": analysis,
        "summary": summary,
        "processing_status": "completed"
    }
    
    writer("文本分析工作流已完成")
    return result

# 测试自定义数据流式传输
config = {"configurable": {"thread_id": "streaming_workflow_1"}}
sample_text = """
人工智能是一个快速发展的领域。它包含机器学习、深度学习、自然语言处理等多个分支。
AI 技术正在改变我们的生活方式。从智能助手到自动驾驶，AI 无处不在。
我们需要负责任地开发和使用这些技术。
"""

print("=== 流式执行文本分析工作流 ===")
print("监听自定义流和更新流...")
print("-" * 50)

for chunk in text_analysis_workflow.stream(
    sample_text, 
    config, 
    stream_mode=["custom", "updates"]
):
    if chunk[0] == "custom":
        print(f"📢 进度更新: {chunk[1]}")
    elif chunk[0] == "updates":
        print(f"🔄 工作流更新: {list(chunk[1].keys())}")

print("-" * 50)
print("流式执行完成！")

# 获取最终结果
final_state = text_analysis_workflow.get_state(config)
if hasattr(final_state, 'values') and final_state.values:
    print("\n=== 最终结果 ===")
    # 注意：实际的结果访问方式可能因版本而异
    print("工作流执行完成，查看流式输出了解详细过程。")

=== 流式执行文本分析工作流 ===
监听自定义流和更新流...
--------------------------------------------------
📢 进度更新: 开始文本分析工作流
📢 进度更新: 输入文本长度: 98 字符
📢 进度更新: 阶段 1: 正在分析文本结构...
🔄 工作流更新: ['analyze_text']
📢 进度更新: 分析完成: 发现 5 个单词
📢 进度更新: 阶段 2: 正在生成摘要...
🔄 工作流更新: ['generate_summary']
📢 进度更新: 摘要生成完成
📢 进度更新: 文本分析工作流已完成
🔄 工作流更新: ['text_analysis_workflow']
--------------------------------------------------
流式执行完成！

=== 最终结果 ===
工作流执行完成，查看流式输出了解详细过程。


**💡 自定义流式传输的应用价值**：

- **进度追踪**: 为用户提供详细的执行进度信息
- **实时反馈**: 长时间运行的任务不再是黑盒操作
- **调试支持**: 通过流式输出更容易定位问题
- **用户体验**: 显著改善用户对系统响应性的感知

##### 示例 6-16：在 Functional API 中实现重试策略

演示更复杂的重试策略配置和错误处理：

In [20]:
from langgraph.types import RetryPolicy
import random

# 定义不同的重试策略
network_retry_policy = RetryPolicy(max_attempts=3, retry_on=ConnectionError)
validation_retry_policy = RetryPolicy(max_attempts=2, retry_on=ValueError)

@task(name="network_request", retry_policy=network_retry_policy)
def make_network_request(url: str) -> dict:
    """模拟网络请求，可能失败的任务。"""
    print(f"尝试访问: {url}")
    
    # 模拟网络不稳定
    if random.random() < 0.7:  # 70% 失败率
        print("❌ 网络连接失败")
        raise ConnectionError(f"无法连接到 {url}")
    
    print("✅ 网络请求成功")
    return {
        "url": url,
        "status": "success",
        "data": f"来自 {url} 的数据"
    }

@task(name="data_validator", retry_policy=validation_retry_policy)
def validate_response_data(data: dict) -> dict:
    """验证响应数据，可能需要重试的任务。"""
    print(f"验证数据: {list(data.keys())}")
    
    # 模拟验证逻辑
    if "status" not in data:
        print("❌ 数据验证失败：缺少 status 字段")
        raise ValueError("响应数据格式不正确")
    
    if data.get("status") != "success":
        print("❌ 数据验证失败：状态不是 success")
        raise ValueError("响应状态异常")
    
    print("✅ 数据验证通过")
    return {
        "validated": True,
        "original_data": data
    }

@task
def process_validated_data(validated_data: dict) -> str:
    """处理已验证的数据。"""
    original = validated_data["original_data"]
    return f"处理完成: {original['data']}"

@entrypoint(checkpointer=MemorySaver())
def robust_api_workflow(api_url: str) -> dict:
    """具有重试机制的健壮 API 工作流。"""
    try:
        print(f"=== 开始处理 API 请求: {api_url} ===")
        
        # 步骤 1: 网络请求（带重试）
        print("步骤 1: 发起网络请求")
        response_data = make_network_request(api_url).result()
        
        # 步骤 2: 数据验证（带重试）
        print("\n步骤 2: 验证响应数据")
        validated_data = validate_response_data(response_data).result()
        
        # 步骤 3: 处理数据（无重试）
        print("\n步骤 3: 处理验证数据")
        processed_result = process_validated_data(validated_data).result()
        
        return {
            "status": "success",
            "result": processed_result,
            "api_url": api_url
        }
        
    except ConnectionError as e:
        print(f"\n💥 网络错误（重试后仍失败）: {e}")
        return {
            "status": "network_error",
            "error": str(e),
            "api_url": api_url
        }
        
    except ValueError as e:
        print(f"\n💥 数据验证错误（重试后仍失败）: {e}")
        return {
            "status": "validation_error",
            "error": str(e),
            "api_url": api_url
        }
        
    except Exception as e:
        print(f"\n💥 未预期错误: {e}")
        return {
            "status": "unexpected_error",
            "error": str(e),
            "api_url": api_url
        }

# 测试重试机制
api_endpoints = [
    "https://api.example.com/data",
    "https://api.service.com/info",
    "https://api.test.com/status"
]

for i, endpoint in enumerate(api_endpoints, 1):
    print(f"\n{'='*60}")
    print(f"测试 {i}: {endpoint}")
    print(f"{'='*60}")
    
    config = {"configurable": {"thread_id": f"robust_api_workflow_{i}"}}
    result = robust_api_workflow.invoke(endpoint, config)
    
    print("\n📊 最终结果:")
    print(f"状态: {result['status']}")
    if result['status'] == 'success':
        print(f"处理结果: {result['result']}")
    else:
        print(f"错误信息: {result['error']}")
    
    time.sleep(0.5)  # 给用户时间观察输出


测试 1: https://api.example.com/data
=== 开始处理 API 请求: https://api.example.com/data ===
步骤 1: 发起网络请求
尝试访问: https://api.example.com/data
❌ 网络连接失败
尝试访问: https://api.example.com/data
❌ 网络连接失败
尝试访问: https://api.example.com/data
❌ 网络连接失败

💥 网络错误（重试后仍失败）: 无法连接到 https://api.example.com/data

📊 最终结果:
状态: network_error
错误信息: 无法连接到 https://api.example.com/data

测试 2: https://api.service.com/info
=== 开始处理 API 请求: https://api.service.com/info ===
步骤 1: 发起网络请求
尝试访问: https://api.service.com/info
❌ 网络连接失败
尝试访问: https://api.service.com/info
❌ 网络连接失败
尝试访问: https://api.service.com/info
❌ 网络连接失败

💥 网络错误（重试后仍失败）: 无法连接到 https://api.service.com/info

📊 最终结果:
状态: network_error
错误信息: 无法连接到 https://api.service.com/info

测试 3: https://api.test.com/status
=== 开始处理 API 请求: https://api.test.com/status ===
步骤 1: 发起网络请求
尝试访问: https://api.test.com/status
❌ 网络连接失败
尝试访问: https://api.test.com/status
❌ 网络连接失败
尝试访问: https://api.test.com/status
✅ 网络请求成功

步骤 2: 验证响应数据
验证数据: ['url', 'status', 'data']
✅ 数据验证通过

步骤 3: 处理验证数据

📊 

**💡 重试策略的重要性**：

- **提高可靠性**: 自动处理临时性错误，减少系统故障
- **区分异常类型**: 不同类型的错误采用不同的重试策略
- **优雅降级**: 重试失败后提供有意义的错误信息
- **生产就绪**: 是构建健壮生产系统的必要功能

##### 示例 6-17：在 Functional API 中实现短期记忆

展示如何使用 `previous` 参数和 `entrypoint.final()` 管理工作流状态：

In [22]:
from typing import Any

@entrypoint(checkpointer=MemorySaver())
def counter_workflow(increment: int, *, previous: Any = None) -> 'entrypoint.final[int, int]':
    """使用先前状态维护计数器的工作流。"""
    current_count = previous if previous is not None else 0
    new_count = current_count + increment
    
    print(f"当前计数: {current_count}, 增量: {increment}, 新计数: {new_count}")
    
    # 返回当前值给调用者，保存新值到状态
    return entrypoint.final(value=current_count, save=new_count)

@entrypoint(checkpointer=MemorySaver())
def session_chat_workflow(
    user_message: str, 
    *, 
    previous: Any = None
) -> 'entrypoint.final[str, dict]':
    """带有会话记忆的聊天工作流。"""
    # 初始化或恢复会话状态
    if previous is None:
        session_state = {
            "messages": [],
            "user_name": None,
            "conversation_count": 0
        }
        print("🆕 开始新的对话会话")
    else:
        session_state = previous.copy()
        print(f"🔄 恢复对话会话 (已有 {len(session_state['messages'])} 条消息)")
    
    # 更新会话状态
    session_state["conversation_count"] += 1
    session_state["messages"].append({"role": "user", "content": user_message})
    
    # 简单的聊天逻辑
    if "我叫" in user_message or "我是" in user_message:
        # 尝试提取用户名
        parts = user_message.replace("我叫", "").replace("我是", "").strip()
        if parts:
            session_state["user_name"] = parts
            response = f"很高兴认识你，{parts}！"
        else:
            response = "很高兴认识你！"
    elif session_state["user_name"]:
        response = f"{session_state['user_name']}，这是我们的第 {session_state['conversation_count']} 次交流。你说：{user_message}"
    else:
        response = f"这是我们的第 {session_state['conversation_count']} 次交流。你好！"
    
    # 添加助手回复到会话历史
    session_state["messages"].append({"role": "assistant", "content": response})
    
    return entrypoint.final(value=response, save=session_state)

# 测试计数器工作流
print("=== 测试计数器工作流 ===")
config_counter = {"configurable": {"thread_id": "counter_workflow_1"}}

for i, increment in enumerate([1, 2, 3, 5], 1):
    result = counter_workflow.invoke(increment, config_counter)
    print(f"调用 {i}: 增量 {increment} -> 返回值 {result}")
print()

# 测试会话聊天工作流
print("=== 测试会话聊天工作流 ===")
config_chat = {"configurable": {"thread_id": "chat_workflow_1"}}

conversation = [
    "你好！",
    "我叫小明",
    "今天天气怎么样？",
    "我喜欢编程",
    "再见！"
]

for i, message in enumerate(conversation, 1):
    print(f"\n--- 对话轮次 {i} ---")
    print(f"👤 用户: {message}")
    response = session_chat_workflow.invoke(message, config_chat)
    print(f"🤖 助手: {response}")

# 查看最终会话状态
print("\n=== 查看最终会话状态 ===")
final_state = session_chat_workflow.get_state(config_chat)
if hasattr(final_state, 'values'):
    print("会话已保存，包含详细的对话历史")
else:
    print("状态管理正常工作")

=== 测试计数器工作流 ===
当前计数: 0, 增量: 1, 新计数: 1
调用 1: 增量 1 -> 返回值 0
当前计数: 1, 增量: 2, 新计数: 3
调用 2: 增量 2 -> 返回值 1
当前计数: 3, 增量: 3, 新计数: 6
调用 3: 增量 3 -> 返回值 3
当前计数: 6, 增量: 5, 新计数: 11
调用 4: 增量 5 -> 返回值 6

=== 测试会话聊天工作流 ===

--- 对话轮次 1 ---
👤 用户: 你好！
🆕 开始新的对话会话
🤖 助手: 这是我们的第 1 次交流。你好！

--- 对话轮次 2 ---
👤 用户: 我叫小明
🔄 恢复对话会话 (已有 2 条消息)
🤖 助手: 很高兴认识你，小明！

--- 对话轮次 3 ---
👤 用户: 今天天气怎么样？
🔄 恢复对话会话 (已有 4 条消息)
🤖 助手: 小明，这是我们的第 3 次交流。你说：今天天气怎么样？

--- 对话轮次 4 ---
👤 用户: 我喜欢编程
🔄 恢复对话会话 (已有 6 条消息)
🤖 助手: 小明，这是我们的第 4 次交流。你说：我喜欢编程

--- 对话轮次 5 ---
👤 用户: 再见！
🔄 恢复对话会话 (已有 8 条消息)
🤖 助手: 小明，这是我们的第 5 次交流。你说：再见！

=== 查看最终会话状态 ===
会话已保存，包含详细的对话历史


**💡 短期记忆机制解析**：

- **`previous` 参数**: 自动注入上次执行保存的状态
- **`entrypoint.final()`**: 分离返回值和保存状态，提供灵活的状态管理
- **状态持续性**: 同一 `thread_id` 下的状态在调用间保持连续
- **应用场景**: 适用于需要跨调用维护上下文的对话系统

##### 示例 6-18：在 Functional API 中实现长期记忆

In [27]:
from langgraph.func import entrypoint, task
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.store.base import BaseStore
from langgraph.types import StreamWriter
from langchain_core.runnables import RunnableConfig
from typing import Any

# 初始化长期内存存储
store = InMemoryStore()

@task
def retrieve_user_preferences(user_id: str, *, store: BaseStore):
    """从长期内存中检索用户偏好的任务"""
    try:
        preferences_data = store.get(("user_preferences", user_id), "preferences")
        if preferences_data:
            return preferences_data.value
        else:
            return {"theme": "light", "language": "zh-CN", "notifications": True}  # 默认偏好
    except Exception as e:
        print(f"获取用户偏好时出错: {e}")
        return {"theme": "light", "language": "zh-CN", "notifications": True}

@task  
def store_user_preferences(user_id: str, preferences: dict, *, store: BaseStore):
    """存储用户偏好到长期内存的任务"""
    try:
        store.put(("user_preferences", user_id), "preferences", preferences)
        return f"已保存用户 {user_id} 的偏好设置"
    except Exception as e:
        print(f"存储用户偏好时出错: {e}")
        return f"保存失败: {e}"

@entrypoint(checkpointer=MemorySaver(), store=store)
def personalized_workflow(
    input_data: dict,  
    *,
    store: BaseStore,
    writer: StreamWriter,
    config: RunnableConfig
) -> str:
    """访问长期记忆的个性化工作流"""
    # 从输入数据中提取参数
    user_id = input_data["user_id"]
    query = input_data["query"]

    thread_id = config.get("configurable", {}).get("thread_id", "unknown")
    writer(f"开始个性化工作流，线程: {thread_id}")

    # 获取用户偏好
    writer(f"获取用户 {user_id} 的偏好...")
    user_prefs = retrieve_user_preferences(user_id, store=store).result()
    writer(f"用户偏好: {user_prefs}")

    # 根据偏好处理查询
    if user_prefs.get("language") == "zh-CN":
        response = f"您好！根据您的查询 '{query}' 和偏好设置，为您提供中文服务。"
    else:
        response = f"Hello! Based on your query '{query}' and preferences, providing service in English."

    # 可选：更新偏好（例如记录最近查询）
    updated_prefs = user_prefs.copy()
    updated_prefs["last_query"] = query
    store_result = store_user_preferences(user_id, updated_prefs, store=store).result()
    writer(f"偏好更新结果: {store_result}")

    writer("个性化工作流完成")
    return response


config = {"configurable": {"thread_id": "personalized_workflow_1"}}

# 方式1: 使用 invoke 同步执行
print("=== 使用 invoke 同步执行 ===")
input_data = {
    "user_id": "user_123",
    "query": "今天天气怎么样？"
}
result = personalized_workflow.invoke(input_data, config=config)
print(f"结果: {result}")
print()

# 方式2: 使用 stream 流式执行（可以看到中间过程）
print("=== 使用 stream 流式执行 ===")
input_data2 = {
    "user_id": "user_456",
    "query": "What's the weather like today?"
}
for chunk in personalized_workflow.stream(
    input_data2,
    config={"configurable": {"thread_id": "personalized_workflow_2"}},
    stream_mode=["custom", "updates"]
):
    if chunk[0] == "custom":
        print(f"📢 进度: {chunk[1]}")
    elif chunk[0] == "updates":
        print(f"🔄 更新: {list(chunk[1].keys())}")
print()

# 方式3: 测试存储持久性 - 再次调用同一用户
print("=== 测试存储持久性 ===")
input_data3 = {
    "user_id": "user_123",  # 同一用户
    "query": "明天会下雨吗？"
}
result2 = personalized_workflow.invoke(input_data3, config={"configurable": {"thread_id": "personalized_workflow_3"}})
print(f"第二次调用结果: {result2}")

=== 使用 invoke 同步执行 ===
结果: 您好！根据您的查询 '今天天气怎么样？' 和偏好设置，为您提供中文服务。

=== 使用 stream 流式执行 ===
📢 进度: 开始个性化工作流，线程: personalized_workflow_2
📢 进度: 获取用户 user_456 的偏好...
🔄 更新: ['retrieve_user_preferences']
📢 进度: 用户偏好: {'theme': 'light', 'language': 'zh-CN', 'notifications': True}
🔄 更新: ['store_user_preferences']
📢 进度: 偏好更新结果: 已保存用户 user_456 的偏好设置
📢 进度: 个性化工作流完成
🔄 更新: ['personalized_workflow']

=== 测试存储持久性 ===
第二次调用结果: 您好！根据您的查询 '明天会下雨吗？' 和偏好设置，为您提供中文服务。


### API 特性对比总结

| 特性 | `create_react_agent` | Functional API | Graph API |
|------|---------------------|----------------|----------|
| **学习曲线** | 最低 | 中等 | 较高 |
| **开发速度** | 最快 | 快速 | 中等 |
| **灵活性** | 有限 | 高 | 最高 |
| **可扩展性** | 低 | 中等 | 高 |
| **适用场景** | ReAct 模式 | 中等复杂度 | 复杂系统 |
| **代码风格** | 配置式 | 函数式 | 声明式 |
| **调试支持** | LangSmith 自动 | 任务级跟踪 | 节点级可视化 |
| **状态管理** | 自动 | 灵活 | 完全控制 |

## 📚 本章总结

通过本章的学习，我们全面探索了 LangGraph 框架提供的三种主要 API 选项，深入理解了它们各自的特点、优势和适用场景。首先学习了 `create_react_agent` 预构建 API，掌握了快速构建 ReAct 智能体的方法，包括自定义提示、记忆集成、人机环路和结构化输出等高级功能。接着深入研究了 Functional API，学会使用 `@entrypoint` 和 `@task` 装饰器构建函数式工作流，掌握了并行执行、子工作流组合、自定义流式传输、重试策略和状态管理等核心技术。然后回顾了 Graph API 的核心价值，理解了其在处理复杂智能体架构中的不可替代性。最后建立了 API 选择决策框架，学会根据项目复杂度、团队经验、时间要求和可扩展性需求选择最适合的 API。这些多样化的 API 选项确保了 LangGraph 能够满足从快速原型到复杂生产系统的各种开发需求，为构建下一代 AI 智能体应用提供了强有力的工具支持。